In [1]:
import requests
import pandas as pd
import json
from dateutil import parser


In [2]:
API_KEY = "a2faf0838c2aacce44be67cea8b40c06-48d69bac26199a4d5850294881134c34"
ACCOUNT_ID = "101-004-21509763-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [3]:
session = requests.Session()

In [4]:
session.headers.update({
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {API_KEY}'
})

In [5]:
params = dict(
    count = 10,
    granularity = "H1"
)

In [6]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [7]:
response = session.get(url, params=None, data=None, headers=None)

In [8]:
response.status_code

200

In [9]:
data = response.json()

In [10]:
data.keys()

dict_keys(['instruments', 'lastTransactionID'])

In [11]:
instruments_list = data["instruments"]

In [12]:
len(instruments_list)

123

In [13]:
instruments_list[1].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [14]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [15]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = {k: i[k] for k in key_i }

In [16]:
instruments_dict['AU200_AUD']

{'name': 'AU200_AUD',
 'type': 'CFD',
 'displayName': 'Australia 200',
 'pipLocation': 0,
 'displayPrecision': 1,
 'tradeUnitsPrecision': 1,
 'marginRate': '0.05'}

In [18]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

FileNotFoundError: [Errno 2] No such file or directory: '../data/instruments.json'

In [ ]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data
    

In [ ]:

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        prices = ['mid', 'bid', 'ask']
        ohlc = ['o', 'h', 'l', 'c']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
            
        final_data.append(new_dict)
    df = pd.DataFrame(final_data)
    return df


In [ ]:
def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name)
    if len(data) == 0:
        print("No candles there", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"Pair name:{pair_name}, Granularity: {granularity}, Candle shape: {candles_df.shape[0]}, time: {candles_df.time.min()}")

In [ ]:
code, data = fetch_candles("EUR_SEK", count=10, granularity="H1")
candles_df = get_candles_df(data)

In [ ]:
create_data_file("EUR_SEK", count=100, granularity="H4")

Pair name:EUR_SEK, Granularity: H4, Candle shape: 99, time: 2023-10-31 05:00:00+00:00


In [ ]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'AUD', 'CAD', 'NZD', 'HKD', 'SEK']

In [ ]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
                for g in ["M15", "M30", "H1", "H2", "H3", "H4"]:
                    create_data_file(pair_name=pr, count=4001, granularity=g)

Pair name:EUR_USD, Granularity: M15, Candle shape: 4000, time: 2023-09-26 01:45:00+00:00
Pair name:EUR_USD, Granularity: M30, Candle shape: 4000, time: 2023-07-28 09:30:00+00:00
Pair name:EUR_USD, Granularity: H1, Candle shape: 4000, time: 2023-04-04 01:00:00+00:00
Pair name:EUR_USD, Granularity: H2, Candle shape: 4000, time: 2022-08-11 07:00:00+00:00
Pair name:EUR_USD, Granularity: H3, Candle shape: 4000, time: 2021-12-21 16:00:00+00:00
Pair name:EUR_USD, Granularity: H4, Candle shape: 4000, time: 2021-05-03 01:00:00+00:00
Pair name:EUR_GBP, Granularity: M15, Candle shape: 4000, time: 2023-09-26 01:45:00+00:00
Pair name:EUR_GBP, Granularity: M30, Candle shape: 4000, time: 2023-07-28 09:30:00+00:00
Pair name:EUR_GBP, Granularity: H1, Candle shape: 4000, time: 2023-04-04 01:00:00+00:00
Pair name:EUR_GBP, Granularity: H2, Candle shape: 4000, time: 2022-08-11 07:00:00+00:00
Pair name:EUR_GBP, Granularity: H3, Candle shape: 4000, time: 2021-12-21 16:00:00+00:00
Pair name:EUR_GBP, Granulari

KeyboardInterrupt: 